In [1]:
#필요한 modules
from IPython.display import Image, display, HTML
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import requests
import os
from datetime import datetime
import pandas as pd
import selenium.webdriver as webdriver
import selenium.webdriver.support.ui as ui
from selenium.webdriver.common.keys import Keys
from time import sleep
from multiprocessing import Process
from multiprocessing import Pool
os.chdir("C:/Users/Lab_/Desktop/crawling") #디렉토리 설정

In [2]:
#Setting options on driver that I use

TEST_URL = 'https://intoli.com/blog/making-chrome-headless-undetectable/chrome-headless-test.html'

options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('window-size=1920x1080')
options.add_argument("disable-gpu")
options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36")
options.add_argument("lang=ko_KR") # 한국어!
driver = webdriver.Chrome('chromedriver', chrome_options=options)

driver.get(TEST_URL)
driver.execute_script("Object.defineProperty(navigator, 'plugins', {get: function() {return[1, 2, 3, 4, 5]}})")
driver.execute_script("Object.defineProperty(navigator, 'languages', {get: function() {return ['ko-KR', 'ko']}})")
driver.execute_script("const getParameter = WebGLRenderingContext.getParameter;WebGLRenderingContext.prototype.getParameter = function(parameter) {if (parameter === 37445) {return 'NVIDIA Corporation'} if (parameter === 37446) {return 'NVIDIA GeForce GTX 980 Ti OpenGL Engine';}return getParameter(parameter);};")

user_agent = driver.find_element_by_css_selector('#user-agent').text
plugins_length = driver.find_element_by_css_selector('#plugins-length').text
languages = driver.find_element_by_css_selector('#languages').text
webgl_vendor = driver.find_element_by_css_selector('#webgl-vendor').text
webgl_renderer = driver.find_element_by_css_selector('#webgl-renderer').text

print('User-Agent: ', user_agent)
print('Plugin length: ', plugins_length)
print('languages: ', languages)
print('WebGL Vendor: ', webgl_vendor)
print('WebGL Renderer: ', webgl_renderer)

User-Agent:  Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36
Plugin length:  0
languages:  ko-KR
WebGL Vendor:  Google Inc.
WebGL Renderer:  Google SwiftShader


In [8]:
f = open("C:/Users/Lab_/Desktop/crawling/total_link_set2.txt", 'r')
a = f.readlines()

In [9]:
b = []
for i in a:
    j = i.replace("\n", "")
    b.append(j)

In [10]:
total_link = b
total_link = total_link[50005:60006]

In [11]:
len(total_link)

10001

In [12]:
# Get scroll height
start_time = time.time()
number = 16
total_id = []
total_text = []
total_tag = []
total_pic = []
total_time = []
total_num = 0



for link in total_link:
    try:
        driver.get("https://www.instagram.com"+link)
            
        html_inside = driver.page_source
        soup_inside = BeautifulSoup(html_inside, 'html.parser')    
                
        Content = soup_inside.select(
            'li._ezgzd')
            
        Times = soup_inside.select(
            'time._p29ma')
            
            
        #ID Section
        ID = (Content[0].find('a', class_='_2g7d5'))
        total_id.append(ID["title"])
        if len(total_id) == 2500 or len(total_id) == 7500:
            print("We are doing great!")
        
        #Date Section
        Time = Times[0]["datetime"]
        total_time.append(Time)
            
        #TAGS & TEXT
        TAGS_TXT = Content[0].find('span')      
        TXTS = TAGS_TXT.findAll('span')
                
        #TEXTS
        TXTS_list = []
        for i in TXTS:
            if i.text != " ":
                TXTS_list.append(i.text)
            
        TXT = " ".join(TXTS_list)
        total_text.append(TXT)
                
        #print(TXT)
                
        #TAGS
        TAGS = TAGS_TXT.findAll('a')
        TAGS_list = []
        for TAG in TAGS:
            TAGS_list.append(TAG.text)
        total_tag.append(TAGS_list)
        #print(TAGS_list)
                
        #PICTURE
        Picture = soup_inside.select(
            "img")
        total_pic.append(Picture[1]['src'])
        #print(Picture[1]['src'])
        
        
    except:
        continue
        
    if len(total_time) == 10001:
            #만개가 넘어가면 엑셀로 반출하고 안에 있는 리스트 초기화
            total_num += len(total_id)
            df = pd.DataFrame({'PIC': total_pic,
                               'TEXT':total_text,
                               'TAGS':total_tag,
                               'ID': total_id,
                               'TIME': total_time
                                })

            df.to_excel('instagram_soldrink_' + str(number) + '.xlsx', sheet_name = 'sheet')

            total_id = []
            total_text = []
            total_tag = []
            total_pic = []
            total_time = []
            print("FLUSHED, LET'S RESTART" + str(number) + '개 파일 생성')
            number += 1
    if number == 16 and link == total_link[-1]:
        total_num += len(total_id)
        df = pd.DataFrame({'PIC': total_pic,
                           'TEXT':total_text,
                           'TAGS':total_tag,
                           'ID': total_id,
                           'TIME': total_time
                            })

        df.to_excel('instagram_soldrink_' + str(number) + '.xlsx', sheet_name = 'sheet')

        
        print("FLUSHED, LET'S RESTART" + str(number) + '개 파일 생성')
        print("Finished, Farewell...")
        break      
            
end_time = time.time()
print(end_time - start_time)

We are doing great!
We are doing great!
FLUSHED, LET'S RESTART16개 파일 생성
Finished, Farewell...
12445.00579714775
